#### Installling the necessary packages

In [543]:
# ! pip install pandas
# ! pip install azure-storage-blob
# ! pip install dotevn


In [544]:
# Importing Neccessary Libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

Data Loading

In [545]:
# Data loading

try:
    data = pd.read_csv(r'raw_data\zipco_transaction.csv')
    print("Data Extracted Successfully")
except Exception as e:
    print(f"An error occured {e}")

Data Extracted Successfully


In [546]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [547]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [548]:
# Convert ate column to datetime
data['Date'] = pd.to_datetime(data['Date'])

In [549]:
# Data Cleaning and transformation
# Remove duplicates

data.drop_duplicates(inplace=True)

In [550]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             900 non-null    float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        900 non-null    object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [551]:
for cols in data:
   print(f"{cols} number of null values is: {data[cols].isnull().sum()}")

Date number of null values is: 0
ProductName number of null values is: 0
Quantity number of null values is: 0
UnitPrice number of null values is: 0
StoreLocation number of null values is: 0
PaymentType number of null values is: 0
PromotionApplied number of null values is: 0
Weather number of null values is: 0
Temperature number of null values is: 100
StaffPerformanceRating number of null values is: 0
CustomerFeedback number of null values is: 100
DeliveryTime_min number of null values is: 0
OrderType number of null values is: 0
CustomerName number of null values is: 0
CustomerAddress number of null values is: 0
Customer_PhoneNumber number of null values is: 0
CustomerEmail number of null values is: 100
Staff_Name number of null values is: 0
Staff_Email number of null values is: 100
DayOfWeek number of null values is: 0
TotalSales number of null values is: 0


HANDLING MISSING VALUES

In [552]:
# REPLACING MISSSING VALUES: METHOD 1     # LESS EFFICIENT

# data['CustomerEmail'] = data['CustomerEmail'].fillna("Unknown")
# data['Temperature'] = data['Temperature'].fillna(data['Temperature'].mean())
# data['CustomerFeedback'] = data['CustomerFeedback'].fillna("Unknown")
# data['Staff_Email'] = data['Staff_Email'].fillna("Unknown")
# data.info()

In [553]:
# REPLACING MISSSING VALUES: METHOD 2
numeric_columns = data.select_dtypes(include=['Float64', 'int64']).columns
for cols in numeric_columns:
      data.fillna({cols:data[cols].mean()},inplace=True)

In [554]:
# Handling missing values(Fill Missing strings/Objects with 'Unknown')
string_columns = data.select_dtypes(include=['object']).columns
for str_cols in string_columns:
    data.fillna({str_cols:'Unknown'}, inplace=True)

In [555]:
data.info()
data.to_csv(r'cleaned_data\cleaned.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [556]:
# product = data[['ProductName','UnitPrice']].copy().drop_duplicates().reset_index(drop=True)
# product_ID = range(1, len(product) + 1)
# product['Product_ID'] = product_ID
#                     OR

product = data[['ProductName']].copy().drop_duplicates().reset_index(drop=True)
product.index.name = 'Product_ID'
product = product.reset_index()


In [557]:
product

,Product_ID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni
5,5,Pizza Margherita
6,6,Pizza Beef Double decker
7,7,Cheesecake
8,8,Fruit Cake


In [558]:
customer = data[['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail']].copy().drop_duplicates().reset_index(drop=True)
customer.index.name = 'Customer_ID'
customer = customer.reset_index()


In [559]:
customer

,Customer_ID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net
...,...,...,...,...,...
995,995,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com
996,996,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com
997,997,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org
998,998,Kathryn Nichols,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org


In [560]:
staff = data[['Staff_Name', 'Staff_Email']].copy().drop_duplicates().reset_index(drop=True)
staff.index.name = 'Staff_ID'
staff = staff.reset_index()

In [561]:
staff.head()

,Staff_ID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [562]:
# fact_table = data[['Date','Quantity','UnitPrice','StoreLocation','PaymentType','PromotionApplied','Weather','Temperature','StaffPerformanceRating','CustomerFeedback','DeliveryTime_min','OrderType','DayOfWeek','TotalSales']]\
#     .join(product, how='inner')\
#     .join(customer, how= 'inner')\
#     .join(staff, how = 'inner')

In [563]:
transcation_fact = data.merge(product, on=['ProductName'], how='left')\
      .merge(customer, on=['CustomerName','CustomerAddress','Customer_PhoneNumber','CustomerEmail'], how='left')\
            .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left')

transcation_fact.index.name = 'Transaction_ID'
transcation_fact.reset_index()

,Transaction_ID,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,...,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales,Product_ID,Customer_ID,Staff_ID
0,0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,...,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608,0,0,0
1,1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,...,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070,1,1,1
2,2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,27.154342,...,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321,2,2,2
3,3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,...,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647,3,3,3
4,4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,...,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2023-02-11 11:00:00,Vanilla Cake,2,11.224568,North,Credit Card,True,Rainy,28.787946,...,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com,Lauren Hines,robertmaxwell@example.com,Saturday,22.449136,0,995,995
996,996,2023-02-11 12:00:00,Carrot Cake,3,15.691665,East,Credit Card,False,Cloudy,34.641175,...,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com,Roy Garcia,johnsonangela@example.org,Saturday,47.074995,3,996,996
997,997,2023-02-11 13:00:00,Cheesecake,3,10.793180,West,Cash,True,Sunny,27.984565,...,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org,Bruce Chung,lori61@example.org,Saturday,32.379540,7,997,997
998,998,2023-02-11 14:00:00,Cheesecake,5,13.827574,East,Online,True,Rainy,33.579827,...,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org,Leslie Carter,zreed@example.com,Saturday,69.137870,7,998,998


In [564]:
transcation_fact = transcation_fact[['Customer_ID','Product_ID','Staff_ID','Date', 'Quantity', 'UnitPrice', 'StoreLocation', 'PaymentType',
       'PromotionApplied', 'Weather', 'Temperature', 'StaffPerformanceRating',
       'CustomerFeedback', 'DeliveryTime_min', 'OrderType', 'DayOfWeek',
       'TotalSales', 'ProductName','CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail','Staff_Name', 'Staff_Email']]
transcation_fact


,Customer_ID,Product_ID,Staff_ID,Date,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,...,OrderType,DayOfWeek,TotalSales,ProductName,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email
Transaction_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,2023-01-01 00:00:00,2,12.532304,South,Cash,True,Rainy,...,In Store,Sunday,25.064608,Vanilla Cake,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com
1,1,1,1,2023-01-01 01:00:00,1,7.083070,South,Cash,False,Rainy,...,In Store,Sunday,7.083070,Red Velvet Cake,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net
2,2,2,2,2023-01-01 02:00:00,5,6.736064,North,Cash,True,Rainy,...,Phone Order,Sunday,33.680321,Chocolate Cake,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net
3,3,3,3,2023-01-01 03:00:00,2,7.314823,North,Cash,False,Cloudy,...,Online Order,Sunday,14.629647,Carrot Cake,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org
4,4,4,4,2023-01-01 04:00:00,1,7.577727,East,Credit Card,True,Cloudy,...,In Store,Sunday,7.577727,Pizza Pepperoni,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0,995,2023-02-11 11:00:00,2,11.224568,North,Credit Card,True,Rainy,...,In Store,Saturday,22.449136,Vanilla Cake,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com,Lauren Hines,robertmaxwell@example.com
996,996,3,996,2023-02-11 12:00:00,3,15.691665,East,Credit Card,False,Cloudy,...,In Store,Saturday,47.074995,Carrot Cake,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com,Roy Garcia,johnsonangela@example.org
997,997,7,997,2023-02-11 13:00:00,3,10.793180,West,Cash,True,Sunny,...,Phone Order,Saturday,32.379540,Cheesecake,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org,Bruce Chung,lori61@example.org


In [565]:
transcation_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Customer_ID             1000 non-null   int64         
 1   Product_ID              1000 non-null   int64         
 2   Staff_ID                1000 non-null   int64         
 3   Date                    1000 non-null   datetime64[ns]
 4   Quantity                1000 non-null   int64         
 5   UnitPrice               1000 non-null   float64       
 6   StoreLocation           1000 non-null   object        
 7   PaymentType             1000 non-null   object        
 8   PromotionApplied        1000 non-null   bool          
 9   Weather                 1000 non-null   object        
 10  Temperature             1000 non-null   float64       
 11  StaffPerformanceRating  1000 non-null   object        
 12  CustomerFeedback        1000 non-null   object   

Saving data as CSV

In [566]:
# Save data as CSV files

product.to_csv(r'cleaned_data\products.csv',index=False)
customer.to_csv(r'cleaned_data\customer.csv',index=False)
staff.to_csv(r'cleaned_data\staff.csv',index=False)
transcation_fact.to_csv(r'cleaned_data\transaction_fact.csv',index=False)


Loading data into Azure blob storae

In [567]:
# Data Loading
# Load the environmental variables from .env files
load_dotenv()

connect_str = os.getenv("AZURE_CONNECTION_STRING_VALUE")
container_name = os.getenv("CONTAINER_NAME")

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# blob_service_client = BlobServiceClient.from_connection_string(connect_str).get_container_client(container_name)

# Load data to Azure Blob Storage
# Using tuples to store the data

files = [
    (data, r"raw_data\cleaned.csv"),
    (customer, r"cleaned_data\customer.csv"),
    (product, r"cleaned_data\products.csv"),
    (staff, r"cleaned_data\staff.csv"),
    (transcation_fact, r"cleaned_data\transaction_fact.csv")
]

for file, blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into Azure Blob Storage')


raw_data\cleaned.csv loaded into Azure Blob Storage
cleaned_data\customer.csv loaded into Azure Blob Storage
cleaned_data\products.csv loaded into Azure Blob Storage
cleaned_data\staff.csv loaded into Azure Blob Storage
cleaned_data\transaction_fact.csv loaded into Azure Blob Storage


In [571]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str).get_container_client(container_name)
